In [57]:
!pip install mlxtend

Defaulting to user installation because normal site-packages is not writeable


In [60]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

In [88]:
data = pd.read_csv("nyc_taxi_trip_duration.csv")
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id1080784,2,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,N,400
1,id0889885,1,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,N,1100
2,id0857912,2,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,N,1635
3,id3744273,2,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,N,1141
4,id0232939,1,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,N,848


In [89]:
data.tail()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
729317,id3905982,2,2016-05-21 13:29:38,2016-05-21 13:34:34,2,-73.965919,40.789780,-73.952637,40.789181,N,296
729318,id0102861,1,2016-02-22 00:43:11,2016-02-22 00:48:26,1,-73.996666,40.737434,-74.001320,40.731911,N,315
729319,id0439699,1,2016-04-15 18:56:48,2016-04-15 19:08:01,1,-73.997849,40.761696,-74.001488,40.741207,N,673
729320,id2078912,1,2016-06-19 09:50:47,2016-06-19 09:58:14,1,-74.006706,40.708244,-74.013550,40.713814,N,447
729321,id1053441,2,2016-01-01 17:24:16,2016-01-01 17:44:40,4,-74.003342,40.743839,-73.945847,40.712841,N,1224


In [90]:
data.shape

(729322, 11)

In [91]:

data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime']).astype('int64') // 10**9
data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime']).astype('int64') // 10**9

In [92]:
#drop id column

data = data.drop("id", axis=1)

In [93]:
# categorical data
data["store_and_fwd_flag"] = data["store_and_fwd_flag"].apply(lambda x: 1 if x == "Y" else 0)
data.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,2,1456764021,1456764421,1,-73.953918,40.778873,-73.963875,40.771164,0,400
1,1,1457739337,1457740437,2,-73.988312,40.731743,-73.994751,40.694931,0,1100
2,2,1456077573,1456079208,2,-73.997314,40.721458,-73.948029,40.774918,0,1635
3,2,1451987071,1451988212,6,-73.961670,40.759720,-73.956779,40.780628,0,1141
4,1,1455691343,1455692191,1,-74.017120,40.708469,-73.988182,40.740631,0,848


In [94]:
data.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,2,1456764021,1456764421,1,-73.953918,40.778873,-73.963875,40.771164,0,400
1,1,1457739337,1457740437,2,-73.988312,40.731743,-73.994751,40.694931,0,1100
2,2,1456077573,1456079208,2,-73.997314,40.721458,-73.948029,40.774918,0,1635
3,2,1451987071,1451988212,6,-73.961670,40.759720,-73.956779,40.780628,0,1141
4,1,1455691343,1455692191,1,-74.017120,40.708469,-73.988182,40.740631,0,848


- Missing value ratio
- Low variance 
- High correlation 
- Backward feature elimination
- Forward feature selection 


In [95]:
data.dtypes

vendor_id               int64
pickup_datetime         int64
dropoff_datetime        int64
passenger_count         int64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag      int64
trip_duration           int64
dtype: object

In [96]:
data_SMR = data.copy()
data_low_var = data.copy()
data_high_cor = data.copy()
data_bfe = data.copy()
data_ffs = data.copy()
data_low_var.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,2,1456764021,1456764421,1,-73.953918,40.778873,-73.963875,40.771164,0,400
1,1,1457739337,1457740437,2,-73.988312,40.731743,-73.994751,40.694931,0,1100
2,2,1456077573,1456079208,2,-73.997314,40.721458,-73.948029,40.774918,0,1635
3,2,1451987071,1451988212,6,-73.961670,40.759720,-73.956779,40.780628,0,1141
4,1,1455691343,1455692191,1,-74.017120,40.708469,-73.988182,40.740631,0,848


# missing value 
- no missing values found

In [97]:
data_SMR.isnull().sum()/len(data)*100

vendor_id             0.0
pickup_datetime       0.0
dropoff_datetime      0.0
passenger_count       0.0
pickup_longitude      0.0
pickup_latitude       0.0
dropoff_longitude     0.0
dropoff_latitude      0.0
store_and_fwd_flag    0.0
trip_duration         0.0
dtype: float64

# Low variance

In [98]:
data_low_var.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,2,1456764021,1456764421,1,-73.953918,40.778873,-73.963875,40.771164,0,400
1,1,1457739337,1457740437,2,-73.988312,40.731743,-73.994751,40.694931,0,1100
2,2,1456077573,1456079208,2,-73.997314,40.721458,-73.948029,40.774918,0,1635
3,2,1451987071,1451988212,6,-73.961670,40.759720,-73.956779,40.780628,0,1141
4,1,1455691343,1455692191,1,-74.017120,40.708469,-73.988182,40.740631,0,848


In [99]:
data_low_var.shape

(729322, 10)

In [100]:
data.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,2,1456764021,1456764421,1,-73.953918,40.778873,-73.963875,40.771164,0,400
1,1,1457739337,1457740437,2,-73.988312,40.731743,-73.994751,40.694931,0,1100
2,2,1456077573,1456079208,2,-73.997314,40.721458,-73.948029,40.774918,0,1635
3,2,1451987071,1451988212,6,-73.961670,40.759720,-73.956779,40.780628,0,1141
4,1,1455691343,1455692191,1,-74.017120,40.708469,-73.988182,40.740631,0,848


In [113]:
normalized = normalize(data_low_var)

In [114]:
data_scaled = pd.DataFrame(normalized)

In [115]:
data_scaled.var()

0    5.839887e-20
1    8.797602e-13
2    8.784407e-13
3    4.043547e-19
4    1.313768e-20
5    3.898084e-21
6    1.313108e-20
7    3.939287e-21
8    1.293005e-21
9    3.516401e-12
dtype: float64

In [116]:
variance = data_scaled.var()
columns = data.columns

In [117]:
variable = [ ]
for i in range(0,len(variance)):
    if variance[i]<=2:          
        variable.append(columns[i])

In [118]:
variable

['vendor_id',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'store_and_fwd_flag',
 'trip_duration']

In [119]:
missing_cols = [col for col in variable if col not in data_low_var.columns]
print("Missing columns:", missing_cols)

Missing columns: []


In [120]:
new_data_low_var = data[variable]

In [121]:
new_data_low_var.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,2,1456764021,1456764421,1,-73.953918,40.778873,-73.963875,40.771164,0,400
1,1,1457739337,1457740437,2,-73.988312,40.731743,-73.994751,40.694931,0,1100
2,2,1456077573,1456079208,2,-73.997314,40.721458,-73.948029,40.774918,0,1635
3,2,1451987071,1451988212,6,-73.961670,40.759720,-73.956779,40.780628,0,1141
4,1,1455691343,1455692191,1,-74.017120,40.708469,-73.988182,40.740631,0,848


# High correlation

In [122]:
#Calculate the correlation matrix of the dataset.

In [123]:
data_high_cor.corr()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
vendor_id,1.000000,-0.005176,-0.005152,0.286463,0.008198,0.002861,0.002371,0.005259,-0.080120,0.027752
pickup_datetime,-0.005176,1.000000,1.000000,-0.001131,0.003791,-0.001185,0.003647,-0.001553,0.000511,0.009432
dropoff_datetime,-0.005152,1.000000,1.000000,-0.001120,0.003822,-0.001218,0.003665,-0.001577,0.000514,0.010300
passenger_count,0.286463,-0.001131,-0.001120,1.000000,0.001163,-0.004696,-0.000027,-0.003944,-0.020898,0.013022
pickup_longitude,0.008198,0.003791,0.003822,0.001163,1.000000,0.047635,0.780647,0.118473,0.011732,0.035447
pickup_latitude,0.002861,-0.001185,-0.001218,-0.004696,0.047635,1.000000,0.119970,0.479352,-0.009230,-0.038163
dropoff_longitude,0.002371,0.003647,0.003665,-0.000027,0.780647,0.119970,1.000000,0.149598,0.009801,0.020664
dropoff_latitude,0.005259,-0.001553,-0.001577,-0.003944,0.118473,0.479352,0.149598,1.000000,-0.011415,-0.028283
store_and_fwd_flag,-0.080120,0.000511,0.000514,-0.020898,0.011732,-0.009230,0.009801,-0.011415,1.000000,0.002754
trip_duration,0.027752,0.009432,0.010300,0.013022,0.035447,-0.038163,0.020664,-0.028283,0.002754,1.000000


In [124]:

corr_matrix = data_high_cor.corr().abs()

In [125]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

C:\Users\tlabo\AppData\Local\Temp\ipykernel_6972\2665302050.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [126]:
upper

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
vendor_id,NaN,0.005176,0.005152,0.286463,0.008198,0.002861,0.002371,0.005259,0.080120,0.027752
pickup_datetime,NaN,NaN,1.000000,0.001131,0.003791,0.001185,0.003647,0.001553,0.000511,0.009432
dropoff_datetime,NaN,NaN,NaN,0.001120,0.003822,0.001218,0.003665,0.001577,0.000514,0.010300
passenger_count,NaN,NaN,NaN,NaN,0.001163,0.004696,0.000027,0.003944,0.020898,0.013022
pickup_longitude,NaN,NaN,NaN,NaN,NaN,0.047635,0.780647,0.118473,0.011732,0.035447
pickup_latitude,NaN,NaN,NaN,NaN,NaN,NaN,0.119970,0.479352,0.009230,0.038163
dropoff_longitude,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.149598,0.009801,0.020664
dropoff_latitude,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.011415,0.028283
store_and_fwd_flag,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002754
trip_duration,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [127]:
#==========Identify the pairs of columns that have a 
#========correlation coefficient greater than athreshold

to_drop = [column for column in upper.columns if any(upper[column] > 0.6)]

to_drop


['dropoff_datetime', 'dropoff_longitude']

In [128]:
new_data_high_cor = data_high_cor.drop(data_high_cor[to_drop], axis=1)

In [129]:
new_data_high_cor.head()

,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,2,1456764021,1,-73.953918,40.778873,40.771164,0,400
1,1,1457739337,2,-73.988312,40.731743,40.694931,0,1100
2,2,1456077573,2,-73.997314,40.721458,40.774918,0,1635
3,2,1451987071,6,-73.961670,40.759720,40.780628,0,1141
4,1,1455691343,1,-74.017120,40.708469,40.740631,0,848


In [130]:
new_data_high_cor.shape , data_high_cor.shape

((729322, 8), (729322, 10))

In [131]:
new_data_high_cor.var()

vendor_id             2.487470e-01
pickup_datetime       1.984155e+13
passenger_count       1.722513e+00
pickup_longitude      4.865598e-03
pickup_latitude       1.128565e-03
dropoff_latitude      1.298681e-03
store_and_fwd_flag    5.508714e-03
trip_duration         1.493534e+07
dtype: float64

In [132]:
data_bfe["trip_duration"].quantile(0.25) , data_bfe["trip_duration"].quantile(0.75)
IQR = data_bfe["trip_duration"].quantile(0.75) -data_bfe["trip_duration"].quantile(0.25) 
q1 = data_bfe["trip_duration"].quantile(0.25)
q3 = data_bfe["trip_duration"].quantile(0.75)

lower_wh1 = q1 - (1.5*IQR)
upper_wh2 = q3 + (1.5*IQR)

lower_wh1, upper_wh2
# deleting outliers
data_bfe = data_bfe.drop(data_bfe[data_bfe['trip_duration']>2092].index)

                       



In [133]:
data.loc[data['trip_duration']>2092]

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
15,2,1456019774,1456105938,1,-74.002876,40.734131,-73.955399,40.767502,0,86164
26,1,1466952605,1466954955,2,-74.000168,40.758141,-73.947266,40.764793,0,2350
42,1,1456936180,1456938369,1,-73.997459,40.724735,-73.954414,40.669811,0,2189
55,1,1464649440,1464651720,1,-73.984367,40.760208,-73.913246,40.686783,0,2280
74,2,1464974989,1464977140,1,-74.006973,40.675991,-73.949379,40.717659,0,2151
...,...,...,...,...,...,...,...,...,...,...
729221,2,1454022877,1454025544,1,-73.979088,40.755871,-73.949471,40.594769,0,2667
729232,2,1466210601,1466213883,1,-73.999741,40.728352,-73.964348,40.760529,0,3282
729273,2,1460371079,1460374305,2,-73.782272,40.644493,-74.009514,40.713406,0,3226
729276,1,1465499554,1465502039,1,-73.964828,40.757729,-73.790283,40.643536,0,2485


In [134]:
data_bfe.loc[data_bfe['trip_duration']>2092]

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration


#  Backward feature elimination

In [135]:
# Split the data into features (X) and target (y)
x = data_bfe.drop('trip_duration', axis=1)
y = data_bfe['trip_duration']
x.shape, y.shape

((692359, 9), (692359,))

In [145]:
from mlxtend.feature_selection import SequentialFeatureSelector as seq_feat_s
from sklearn.linear_model import LinearRegression

In [146]:
linear = LinearRegression()

In [147]:

seq_feat_sel = seq_feat_s(estimator=linear,  k_features=5, 
                    forward=False, 
                    scoring='r2')
seq_feat_sel

SequentialFeatureSelector(estimator=LinearRegression(), forward=False,
                          k_features=5, scoring='r2')

In [148]:
seq_feat_sel = seq_feat_sel.fit(x,y)

In [149]:
seq_feat_sel

SequentialFeatureSelector(estimator=LinearRegression(), forward=False,
                          k_features=(5, 5), scoring='r2')

In [150]:
print(dir(seq_feat_sel))


['_TESTING_INTERRUPT_MODE', '__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_calc_confidence', '_check_feature_names', '_check_fitted', '_check_n_features', '_feature_selector', '_get_param_names', '_get_params', '_get_tags', '_more_tags', '_replace_estimator', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_set_params', '_validate_data', '_validate_names', 'clone_estimator', 'cv', 'est_', 'estimator', 'feature_groups', 'feature_groups_', 'feature_names', 'feature_names_to_idx_mapper', 'finalize_fit', 'fit', 'fit_transform', 'fitted', 'fixed_features', 'fixed_features_', 'fix

In [151]:
selected_features = x.columns[list(seq_feat_sel.k_feature_idx_)]
selected_features

Index(['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'pickup_longitude'],
      dtype='object')

In [152]:
new_data_bfe = data_bfe[selected_features]
new_data_bfe.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude
0,2,1456764021,1456764421,1,-73.953918
1,1,1457739337,1457740437,2,-73.988312
2,2,1456077573,1456079208,2,-73.997314
3,2,1451987071,1451988212,6,-73.961670
4,1,1455691343,1455692191,1,-74.017120


# forward feature selection

In [153]:
x = data_bfe.drop('trip_duration', axis=1)
y = data_bfe['trip_duration']
x.shape, y.shape

((692359, 9), (692359,))

In [154]:
linear_reg = LinearRegression()

In [155]:
seq_feat_sel_ffs = seq_feat_s(estimator=linear_reg,  k_features=5, 
                    forward=True, 
                    scoring='r2')
seq_feat_sel_ffs

SequentialFeatureSelector(estimator=LinearRegression(), k_features=5,
                          scoring='r2')

In [156]:
seq_feat_sel_ffs = seq_feat_sel_ffs.fit(x,y)

In [157]:
selected_featuresf = list(seq_feat_sel_ffs.k_feature_names_)
print(selected_featuresf)


['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'dropoff_latitude']


In [158]:
new_data_ffs =  data_ffs[selected_featuresf]
new_data_ffs.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,dropoff_latitude
0,2,1456764021,1456764421,1,40.771164
1,1,1457739337,1457740437,2,40.694931
2,2,1456077573,1456079208,2,40.774918
3,2,1451987071,1451988212,6,40.780628
4,1,1455691343,1455692191,1,40.740631


In [159]:
new_data_ffs.shape , data_ffs.shape

((729322, 5), (729322, 10))

In [160]:
#df_missing.to_csv('nyc_taxi_missing.csv', index=False)




new_data_low_var.to_csv('nyc_taxi_low_var.csv', index=False)
new_data_high_cor.to_csv('nyc_taxi_high_corr.csv', index=False)
new_data_bfe.to_csv('nyc_taxi_back_elim.csv', index=False)
new_data_ffs.to_csv('nyc_taxi_forward_sel.csv', index=False) 
